<a href="https://colab.research.google.com/github/Yutongzhang20080108/Chinese-text-generation-with-prompt-learning/blob/main/LLM_as_a_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I was inspired by the literature "Juding LLM-as-a-Judge with MT-Bench and Chatbot Arena". I would like to use this method to test the Chinese text generation of GPT-3.5 and text it with ERNIE-3.5

In [ ]:
!pip install openai

Getting the generated text from gpt-3.5 Turbo

In [ ]:
import http.client
import json

theme_list = ["自由","生命","理想","高山","大海"]
from openai import OpenAI
text_list = []
client = OpenAI(
    # This is the default and can be omitted
    api_key = "sk-9qxqayMe8D42SRZ3r4yb4SRjdF0Xd2FwUmavZUJZXSTC6VnJ",
    base_url = "https://api.chatanywhere.tech/v1"
)
for theme in theme_list:
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              f"content": "请围绕{theme}这个主题写一首七言绝句,并且生成的内容只需要包括诗歌主体",
          }
      ],
      model="gpt-3.5-turbo",
  )
  text = chat_completion.choices[0].message.content
  text_list.append(text)
print(text_list)

['红尘浮华逐梦飘，\n诗意如烟如雨飞。\n情绪翩翩飞鸟追，\n文字起舞，心扉开。\n流转篇章思绪涌，\n悠然自得，一世哀。\n寂寞辗转，温暖韵，\n{theme}主题，意蕴怀。', '春风吹拂细雨洒，秋叶飘落散苍茫。\n岁月轮回情难绕，生命永恒短暂长。', '天地广阔梦中游，思绪盘旋化成愁。\n沧海桑田情难寄，岁月如梭忧愁留。', '春风吹动初绿芽，项链垂落珠光华。\n纷纷捧出美图案，尽展绚烂自由花。\n\n秋风吹散树枝黄，秋雨拂过桂花香。\n思念心事何时了，怅望人间独自惆。\n\n冬风凛冽漫长夜，瑞雪纷飞字句凝。\n寒夜书写千年梦，诗篇点点暖人心。', '春风拂过江南园，花开满树绣梅笼。\n春色满园人欢喜，美景温存在指尖。']


Using ERNIE-3.5 to evaluate the quality of the generated text

In [ ]:
api_key = "vk05T4TAIxYy6TU6Pk0c9hMz"
secret_key = "zGlgT0MScaGtvQ1lcUEGRLFEjgvGm08U"
import pandas

import requests
import json

def get_access_token():
    """
    使用 API Key，Secret Key 获取access_token，替换下列示例中的应用API Key、应用Secret Key
    """

    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"

    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")
rate = {}
for text in  text_list:
  url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token=" + get_access_token()

  payload = json.dumps({
      "messages": [
          {
              "role": "user",
              "content": f"你的回答只需要一个数字。请你用以下标准：（评分等级为1到5）：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性；③相关性。生成的唐诗与给定主题是否相关；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等；⑤整体。从整体情况评价生成的唐诗。诗歌为:{text}"
          }
      ],
      "response_format":"json_object"
  })
  headers = {
      'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  print(response.text)

{"id":"as-atumu3frba","object":"chat.completion","created":1712282232,"result":"{\n    \"格式\": 3,\n    \"创新性\": 2,\n    \"相关性\": 3,\n    \"美学\": 4,\n    \"整体\": 3\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":169,"completion_tokens":43,"total_tokens":212}}
{"id":"as-jd8i5fiu3d","object":"chat.completion","created":1712282236,"result":"{\n    \"格式\": 3,\n    \"创新性\": 2,\n    \"相关性\": 2,\n    \"美学\": 3,\n    \"整体\": 3\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":133,"completion_tokens":43,"total_tokens":176}}
{"id":"as-pynkgzu5d7","object":"chat.completion","created":1712282243,"result":"{\n    \"格式\": 3,\n    \"创新性\": 2,\n    \"相关性\": 3,\n    \"美学\": 3,\n    \"整体\": 3\n}\n\n这首诗的格式基本符合唐诗的写作规则，但平仄方面略有欠缺。在创新性方面，诗句并未抄袭现成诗句，但也没有明显的创新点。主题相关性上，诗歌表达了一种对时间和情感流逝的感慨，与给定的主题有一定的关联。美学上，诗歌寓情于物，通过描绘天地、沧海桑田等自然景象，传达了诗人的情感。整体而言，这首诗表现了一定的情感和意境，但仍有提升的空间。","is_truncated":false,"need_clear_history"

In [ ]:
from openai import OpenAI
theme_list = ["自由","生命","理想","高山","大海"]
COT_text_list = []
client = OpenAI(
    # This is the default and can be omitted
    api_key = "sk-9qxqayMe8D42SRZ3r4yb4SRjdF0Xd2FwUmavZUJZXSTC6VnJ",
    base_url = "https://api.chatanywhere.tech/v1"
)
for theme in theme_list:
  chat_completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user","content": f"围绕{theme}这个主题通常有哪些情感"}
            ,{"role":"user","content":"针对上述的第一个情感，具体包含着哪些事物"},
            {"role":"user","content":"请你生成一首有关上述的事物和情感的七言律诗"}]
  )
  text = chat_completion.choices[0].message.content
  COT_text_list.append(text)
print(text_list)

['红尘浮华逐梦飘，\n诗意如烟如雨飞。\n情绪翩翩飞鸟追，\n文字起舞，心扉开。\n流转篇章思绪涌，\n悠然自得，一世哀。\n寂寞辗转，温暖韵，\n{theme}主题，意蕴怀。', '春风吹拂细雨洒，秋叶飘落散苍茫。\n岁月轮回情难绕，生命永恒短暂长。', '天地广阔梦中游，思绪盘旋化成愁。\n沧海桑田情难寄，岁月如梭忧愁留。', '春风吹动初绿芽，项链垂落珠光华。\n纷纷捧出美图案，尽展绚烂自由花。\n\n秋风吹散树枝黄，秋雨拂过桂花香。\n思念心事何时了，怅望人间独自惆。\n\n冬风凛冽漫长夜，瑞雪纷飞字句凝。\n寒夜书写千年梦，诗篇点点暖人心。', '春风拂过江南园，花开满树绣梅笼。\n春色满园人欢喜，美景温存在指尖。']


In [ ]:
for text in  COT_text_list:
  url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token=" + get_access_token()

  payload = json.dumps({
      "messages": [
          {
              "role": "user",
              "content": f"你的回答只需要一个数字。请你用以下标准：（评分等级为1到5）：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性；③相关性。生成的唐诗与给定主题是否相关；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等；⑤整体。从整体情况评价生成的唐诗。诗歌为:{text}"
          }
      ],
      "response_format":"json_object"
  })
  headers = {
      'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  print(response.text)

{"id":"as-xvjmw23xrb","object":"chat.completion","created":1712283171,"result":"{\n    \"格式\": 4,\n    \"创新性\": 3,\n    \"相关性\": 4,\n    \"美学\": 4,\n    \"整体\": 4\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":365,"completion_tokens":43,"total_tokens":408}}
{"id":"as-m8zjyjrar1","object":"chat.completion","created":1712283175,"result":"{\n    \"格式\": 4,\n    \"创新性\": 3,\n    \"相关性\": 4,\n    \"美学\": 4,\n    \"整体\": 4\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":367,"completion_tokens":43,"total_tokens":410}}
{"id":"as-7176245kc1","object":"chat.completion","created":1712283180,"result":"{\n    \"格式\": 4,\n    \"创新性\": 3,\n    \"相关性\": 3,\n    \"美学\": 4,\n    \"整体\": 3\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":149,"completion_tokens":43,"total_tokens":192}}
{"id":"as-rsyuuxknf3","object":"chat.completion","created":17